# 1. Постановка задачи

**Задача:**

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель с максимально большим значением accuracy для задачи классификации, которая выберет подходящий тариф. Нужно довести долю правильных ответов по крайней мере до 0.75. Проверить accuracy на тестовой выборке.

**Описание столбцов**

сalls — количество звонков,  
minutes — суммарная длительность звонков в минутах,  
messages — количество sms-сообщений,  
mb_used — израсходованный интернет-трафик в Мб,  
is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).  

# 2. Загрузка данных

Для задачи классификации понадобятся 3 типа моделей: решающее дерево, случайный лес, логистическая регрессия 

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
def read_data_set() -> pd.DataFrame:
    df = pd.DataFrame
    pth1 = 'datasets/users_behavior.csv'
    pth2 = 'https://code.s3.yandex.net/datasets/users_go.csv'
    try:
        df = pd.read_csv(pth1)
    except OSError as e:
        df = pd.read_csv(pth2)
    return df

In [3]:
df = read_data_set()
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Очевидно, что признаками будут все объекты кроме is_ultra, который будет целевым признаком 

# 2. Деление на обущающую, валидационную и тестовую выборки

Так как клиенты из нашей выборки уже перешли на новые тарифы, "спрятанной" тестовой выборки нет, поделим выборку на обучающую (60%), валидационную(20%) и тестовую (20%) выборки

In [6]:
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345)

Создаем тестовую выборку

In [7]:
df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=12345)

Проверяем выборки

In [8]:
df.shape[0]

3214

In [9]:
df_train.shape[0]/df.shape[0] 

0.5998755444928439

In [10]:
df_valid.shape[0]/df.shape[0] 

0.2000622277535781

In [11]:
df_test.shape[0]/df.shape[0] 

0.2000622277535781

In [12]:
df_train.shape[0] + df_valid.shape[0] + df_test.shape[0]

3214

Выборки подготовлены

# 3. Деление на обущающую, валидационную и тестовую выборки

Циклически меняя гиперпараметры  в решающем дереве, случайном лесе и логистическая регрессия найдем лучшую по accuracy модель 

Подготовим признаки и целевой признак для всех моделей

In [13]:
features_train = df_train.drop('is_ultra', axis= 1)
target_train = df_train.is_ultra

In [14]:
features_valid = df_valid.drop('is_ultra', axis= 1)
target_valid = df_valid.is_ultra

## 3.1 Decesion_Tree

Здесь будем менять максимальную глубину дерева

In [15]:
def decession_tree_accuracy(features_train, target_train, features_valid,target_valid):
    accuracy = 0
    best_accuracy = 0
    best_depth = 0
    df_accuracy=pd.DataFrame(columns= ['depth','accuracy'])
    for i in range (1,6):         
        model = DecisionTreeClassifier(max_depth = i, random_state= 12345)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predictions)
        df_accuracy.loc[i, 'depth'] = i
        df_accuracy.loc[i, 'accuracy'] = accuracy
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_depth = i
    return best_accuracy, best_depth, df_accuracy    

In [16]:
best_accuracy, best_depth, df_accuracy = decession_tree_accuracy(features_train, target_train, features_valid,target_valid)

In [17]:
df_accuracy.head()

,depth,accuracy
1,1,0.754277
2,2,0.782271
3,3,0.785381
4,4,0.77916
5,5,0.77916


In [18]:
best_accuracy, best_depth

(0.7853810264385692, 3)

**Вывод**: На валидационной выборке наибольшая точность при глубине дерева 3

## 3.2 RandomForest

Здесь будем перебирать количество деревьев 

In [19]:
def random_forest_accuracy(features_train, target_train, features_valid, target_valid):
    accuracy = 0
    best_accuracy = 0
    best_st = 0
    df_accuracy=pd.DataFrame(columns= ['n_est','accuracy'])
    for i in range (1,11):         
        model = RandomForestClassifier(n_estimators = i, random_state= 12345)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predictions)
        df_accuracy.loc[i, 'n_est'] = i
        df_accuracy.loc[i, 'accuracy'] = accuracy
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_est = i
    return best_accuracy, best_est, df_accuracy    

In [20]:
best_accuracy, best_est, df_accuracy = random_forest_accuracy(features_train, target_train, features_valid, target_valid)

In [23]:
df_accuracy.head(11)

,n_est,accuracy
1,1,0.710731
2,2,0.763608
3,3,0.738725
4,4,0.771384
5,5,0.749611
6,6,0.780715
7,7,0.768274
8,8,0.782271
9,9,0.772939
10,10,0.785381


In [22]:
best_accuracy, best_est

(0.7853810264385692, 10)

**Вывод**: На валидационной выборке наибольшая точность при глубине дерева 10

## 3.3 LogisticRegression

Здесь будем перебирать алгоритмы ('lbfgs' и 'newton-cholesky' так как для остальных выдает ошибку) и количество итерраций

In [88]:
def logistic_regression_accuracy(features_train, target_train, features_valid, target_valid):
    accuracy = 0
    best_accuracy = 0
    algorithms = ['lbfgs', 'newton-cholesky']
    best_itter = 0
    best_algorithm = 'lbfgs'
    shift = 0
    df_accuracy=pd.DataFrame(columns= ['iter','algorithm','accuracy'])
    for alg in algorithms:
        for i in range (100, 2100, 100):        
            model = LogisticRegression(random_state= 12345, solver= alg, max_iter= i)
            model.fit(features_train, target_train)
            predictions = model.predict(features_valid)
            accuracy = accuracy_score(target_valid, predictions) 
            
            if shift == 0:
                df_accuracy.loc[i, 'accuracy'] = accuracy
                df_accuracy.loc[i, 'iter'] = i
                df_accuracy.loc[i, 'algorithm'] = alg
            else:
                df_accuracy.loc[i+20, 'accuracy'] = accuracy
                df_accuracy.loc[i+20, 'iter'] = i
                df_accuracy.loc[i+20, 'algorithm'] = alg
                
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_itter = i
                best_algorithm = alg
        shift = 1
    return best_accuracy, best_itter, best_algorithm, df_accuracy 

In [89]:
best_accuracy, best_itter, best_algorithm, df_accuracy = (
    logistic_regression_accuracy(features_train, target_train, features_valid, target_valid))

In [96]:
df_accuracy.reset_index(drop= True).head(40)

,iter,algorithm,accuracy
0,100,lbfgs,0.710731
1,200,lbfgs,0.710731
2,300,lbfgs,0.710731
3,400,lbfgs,0.710731
4,500,lbfgs,0.710731
5,600,lbfgs,0.710731
6,700,lbfgs,0.710731
7,800,lbfgs,0.710731
8,900,lbfgs,0.710731
9,1000,lbfgs,0.710731


In [84]:
best_accuracy, best_itter, best_algorithm

(0.7558320373250389, 100, 'newton-cholesky')

**Вывод**: Одинаковую и наибольшую точность в 0.785 на валидационной выборке показали решающее дерево и случайный лес, однако случайный лес медленнее решающего дерева

# 4. Проверка качества модели на тестовой выборке

Проверим качество моделей на тестовых выборках

In [ ]:
features_test = df_test.drop('is_ultra', axis= 1)
target_test = df_test.is_ultra

## 4.1. Решающее дерево

In [100]:
model = DecisionTreeClassifier(max_depth = 3, random_state= 12345)    
model.fit(features_train, target_train)

DecisionTreeClassifier(max_depth=3, random_state=12345)

In [101]:
predictions = model.predict(features_test)

In [103]:
accuracy = accuracy_score(target_test, predictions)
accuracy

0.7791601866251944

## 4.2. Случайный лес

In [104]:
model = RandomForestClassifier(n_estimators = 10, random_state= 12345)
model.fit(features_train, target_train)

RandomForestClassifier(n_estimators=10, random_state=12345)

In [105]:
predictions = model.predict(features_test)

In [106]:
accuracy = accuracy_score(target_test, predictions)
accuracy

0.7807153965785381

## 4.3. Логистическая регрессия

In [107]:
model = LogisticRegression(random_state= 12345, solver= 'newton-cholesky', max_iter= 100)
model.fit(features_train, target_train)

LogisticRegression(random_state=12345, solver='newton-cholesky')

In [108]:
predictions = model.predict(features_test)

In [109]:
accuracy = accuracy_score(target_test, predictions)
accuracy

0.7387247278382582

In [110]:
model = LogisticRegression(random_state= 12345, solver= 'lbfgs', max_iter= 100)
model.fit(features_train, target_train)

LogisticRegression(random_state=12345)

In [111]:
accuracy = accuracy_score(target_test, predictions)
accuracy

0.7387247278382582

**Вывод**: Лучший результат на тестовой выборке показал случайный лес с количеством деревьев 10

# 5. Вывод

## 5.1 Постановка задачи

Построить модель с максимально большим значением accuracy, которая выберет подходящий тариф. Доля правильных ответов должна быть по крайней мере до 0.75. Проверить accuracy на тестовой выборке.

## 5.2 Подготовка выборок и выбор модели

В результате подготовки выборок исходный датасет был разделен на тестовую, валидационную и обучающую

Для задачи классификации были выбраны следующие модели:

    Решающее дерево (DecessionTree)
    Случайный лес (RandomForest)
    Логистическая регрессия (LogisticRegression)
    
В ходе обучения и проверки моделей на валидационной выборке, модели c наилучшими гиперпараметрами показали следующие результаты:

     Решающее дерево (DecessionTree) - 0.785
     Случайный лес (RandomForest)  - 0.785
     Логистическая регрессия (LogisticRegression)  - 0.755

После чего модели были проверены на тестовой выборке при наилучших гиперпараметрах.
Резльтаты:

     Решающее дерево (DecessionTree) - 0.779
     Случайный лес (RandomForest)  - 0.78
     Логистическая регрессия (LogisticRegression)  - 0.738

**Вывод**: Лучший результат на тестовой выборке показал случайный лес с количеством деревьев 10. Наилучшее значение accuracy - 0.78